In [3]:
# !pip install gpytorch

In [1]:
import copy

from datetime import datetime
import pandas as pd
import numpy as np
import pickle

import torch
import gpytorch
from torch.utils.data import TensorDataset, DataLoader
from gpytorch.variational import CholeskyVariationalDistribution, VariationalStrategy
from gpytorch.models import ApproximateGP

from torch.optim.lr_scheduler import StepLR

In [2]:
# import crypten
# import crypten.communicator as comm
# crypten.init()
# torch.set_num_threads(1)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
      """
      This initializes the model with some standard settings including the variational distribution to be used, the number of inducing points
      being used along with the mean and the covariance functions being used. 

      Below ConstantMean refers to the Mean being a constant. This means that the mean learnt will act as an offset of sorts whereas all the 
      variance in the data will be explained by the covariance function. The Mean can also be set to linear where it becomes a linear function
      of the features. 

      A basic RBF kernel has been used here that learns one lengthscale for all the dimensions. ScaleKernel below just imbues the RBF Kernel
      with a scalar scale value that can scale the value of the RBF kernel. 
      """
      variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
      variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
      super(GPModel, self).__init__(variational_strategy)
      self.mean_module = gpytorch.means.ConstantMean()
      self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
      """
      This is akin to the forward pass in Neural Nets in a way. Here the mean and the covariance functions are calculated on some given x.
      Then, using that mean and covariance function, a MultivariateNormal distribtuion is represented. 
      """
      mean_x = self.mean_module(x)
      covar_x = self.covar_module(x)
      temp = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
      return temp

In [5]:
ALICE = 0
BOB = 1

In [6]:
cities = list(range(1000))
n= len(cities)
data = {'Temperature': np.random.normal(24, 3, n),
        'Humidity': np.random.normal(78, 2.5, n),
       }
df = pd.DataFrame(data=data, index=cities)
df["Wind"] = df.Temperature + df.Humidity + np.random.normal(0, 0.1, n) 

In [7]:
train_x = df[["Temperature", "Humidity"]].values
train_y = df.Wind.values
train_x, train_y = torch.Tensor(train_x), torch.Tensor(train_y)

In [8]:
# How to share this?
inducing_points = train_x[:500, :]

In [9]:
train_dataset = TensorDataset(train_x, train_y) 
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)

In [10]:
# model = GPModel(inducing_points=inducing_points)
# # The likelihood is learnt separate from the model. This refers to the epsilon/error term in the GP model formulation. 
# likelihood = gpytorch.likelihoods.GaussianLikelihood()
# # Below checks if the cuda/GPU is avaliable or not. If it is, then the model and likelihood both are transferred to the GPU
# if torch.cuda.is_available():
#     model = model.cuda()
#     likelihood = likelihood.cuda()

In [11]:
# # Now we set both to training mode
# model.train()
# likelihood.train()

# # The use of an optimizer is again very similar to how it is used in neural nets. lr is learning rate.
# optimizer = torch.optim.Adam([
#     {'params': model.parameters()},
#     {'params': likelihood.parameters()},
# ], lr=0.05)

# # Our loss object. We're using the VariationalELBO. This defines the loss function we are trying to optimize
# mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

# num_epochs = 500                  ############ please change this if you want to increase or decrease train time ############ 
# for i in range(num_epochs):
#     count = 0
#     mean_loss = 0
#     # Within each iteration, we will go over each minibatch of data
#     minibatch_iter = train_loader
#     for x_batch, y_batch in minibatch_iter:
        
#         output = model(x_batch)
#         loss = -mll(output, y_batch)
#         print(type(loss))
#         optimizer.zero_grad()
#         loss.backward()
# #         print(loss)
#         optimizer.step()
#         mean_loss += loss.item()
#         count+=1
#     if i%10 == 0:
#       print("Mean Loss for Epoch", i,":", mean_loss/count)    

In [22]:
# # Now we set both to training mode
# model.train()
# likelihood.train()

# # The use of an optimizer is again very similar to how it is used in neural nets. lr is learning rate.
# optimizer = torch.optim.Adam([
#     {'params': model.parameters()},
#     {'params': likelihood.parameters()},
# ], lr=0.05)

# # Our loss object. We're using the VariationalELBO. This defines the loss function we are trying to optimize
# mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))
# num_epochs = 500                  ############ please change this if you want to increase or decrease train time ############ 
# epochs_iter = range(num_epochs)

# # Define training parameters
# learning_rate = 0.001
# # num_epochs = 2
# batch_size = 200
# num_batches = train_x.size(0) // batch_size

# for i in range(num_epochs): 
#     print(f"Epoch {i} in progress:")       

#     for batch in range(num_batches):
#         # define the start and end of the training mini-batch
#         start, end = batch * batch_size, (batch + 1) * batch_size

#         # construct CrypTensors out of training examples / labels
#         x_train_ep = train_x[start:end]
#         y_train_ep = train_y[start:end]
# #         y_train = crypten.cryptensor(y_batch, requires_grad=True)

#         # perform forward pass:
#         output = model.forward(x_train_ep)
#         loss_value = -mll(output, y_train_ep)

# #         set gradients to "zero" 
#         optimizer.zero_grad()

#         # perform backward pass: 
#         loss_value.backward(retain_graph=True)

#         # update parameters
#         optimizer.step()

#         # Print progress every batch:
#         batch_loss = loss_value
#         print(f"\tBatch {(batch + 1)} of {num_batches} Loss {batch_loss.item():.4f}")

In [23]:
import crypten
import crypten.communicator as comm
crypten.init()
torch.set_num_threads(1)

In [24]:
import gpytorch
from gpytorch.variational import CholeskyVariationalDistribution, VariationalStrategy
from gpytorch.models import ApproximateGP
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import VariationalELBO

In [25]:
train_x_enc = crypten.cryptensor(train_x, src=ALICE, requires_grad=True)
train_y_enc = crypten.cryptensor(train_y, src=ALICE, requires_grad=True)

In [26]:
# print(train_x.dim())
# print(train_x[0].shape)model_plaintext = GPModel(inducing_points=inducing_points)

In [27]:
# This doesn't work
# model = crypten.nn.from_pytorch(model_plaintext, torch.empty((1, 2)))

In [28]:
class CrypTenModel2(ApproximateGP):
    def __init__(self, inducing_points):
#         super(CrypTenModel2, self).__init__()
        crypten.nn.Module.__init__(self)
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        ApproximateGP.__init__(self, variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
#         print(type(x))
        mean_x = self.mean_module(x)
#         print(type(mean_x))
#         print(type(x))
#         mean_x_enc = crypten.cryptensor(mean_x)
        covar_x = self.covar_module(x)
#         print(type(covar_x))
#         x_enc = crypten.cryptensor(x)
#         mean_x_enc = self.mean_module(x_enc)
#         covar_x_enc = self.covar_module(x_enc)
#         print(type(covar_x))
#         print(type(covar_x_enc))
        temp = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return temp
    
#     def __call__(self, inputs, prior=False, **kwargs):
#         if inputs.dim() == 1:
#             inputs = inputs.unsqueeze(-1)
#         return self.variational_strategy(inputs, prior=prior, **kwargs)

In [29]:
# class Likelihood_crypten(GaussianLikelihood):
#     def __init__(self):
# #         super(CrypTenModel2, self).__init__()
# #         crypten.nn.Module.__init__(self)
#         GaussianLikelihood.__init__(self)
        
class loss_crypten(VariationalELBO):
    def __init__(self):
#         super(CrypTenModel2, self).__init__()
#         crypten.nn.Module.__init__(self)
        VariationalELBO.__init__(self, likelihood, model, num_data=train_y.size(0))
    def forward(self, variational_dist_f, target, **kwargs):
        loss = VariationalELBO.forward(self, variational_dist_f, target, **kwargs)
#         loss_enc = crypten.cryptensor(loss)
        return loss

In [30]:
# inducing_points_enc = crypten.cryptensor(inducing_points, src=ALICE)

In [31]:
model = CrypTenModel2(inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()
# likelihood = Likelihood_crypten()
mll = loss_crypten()

name variational_params_initialized
buffer tensor(0)
name updated_strategy
buffer tensor(1.)
name active_dims
buffer None
name active_dims
buffer None


In [32]:
# Now we set both to training mode
model.train()
likelihood.train()
mll.train()

loss_crypten unencrypted module

In [33]:
model.encrypt()
# print(type(likelihood))
likelihood.encrypt()
mll.encrypt()

VariationalStrategy unencrypted module
tensor(0)
tensor(1.)
ConstantMean unencrypted module
ScaleKernel unencrypted module
None
None
HomoskedasticNoise unencrypted module
GreaterThan(1.000E-04)
GreaterThan(1.000E-04)
GaussianLikelihood encrypted module
HomoskedasticNoise encrypted module
GreaterThan(1.000E-04)
CrypTenModel2 encrypted module
VariationalStrategy encrypted module
ConstantMean encrypted module
ScaleKernel encrypted module


loss_crypten encrypted module

In [34]:
print(model.encrypted)
print(likelihood.encrypted)
print(mll.encrypted)

True
True
True


In [35]:
# is MultivariateNormal encrypted?
model.forward(train_x)

MultivariateNormal(loc: torch.Size([1000]))

In [36]:
# make this crypten?
# likelihood = gpytorch.likelihoods.GaussianLikelihood()
# print(type(likelihood))

In [37]:
# Replaced x.ndimension() to x.dim() in gpytorch library as MPC Tensor does not support .ndimension but .dim() is an alias of it
# not able to encrypt the covraince because a lot of boolean expressions are used, which are encrypted in cryptensor and cannot be
# evaluated before converting them to plaintext
# MPC Tensor when converted to Lazy Tensor has type changed to Lazt Tensor
# line 46 gaussian_likelihood.py
# liene 61, 69 _approximate_mll.py
# line 19 mean.py
# line 300-302, 322, 369, 372 kernel.py
# line 456 module.py
# line 270 lazy_evaluated_kernal_tensor.py
# line 2242 lazy_tensor.py

In [38]:


# The use of an optimizer is again very similar to how it is used in neural nets. lr is learning rate.
# optimizer = torch.optim.Adam([
#     {'params': model.parameters()},
#     {'params': likelihood.parameters()},
# ], lr=0.05)
crypten_optimizer = crypten.optim.SGD([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.05)
# mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))
# print(type(mll))

In [39]:
# Now we set both to training mode
model.train()
likelihood.train()

# The use of an optimizer is again very similar to how it is used in neural nets. lr is learning rate.
# optimizer = torch.optim.Adam([
#     {'params': model.parameters()},
#     {'params': likelihood.parameters()},
# ], lr=0.05)
crypten_optimizer = crypten.optim.SGD([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.05)
# Our loss object. We're using the VariationalELBO. This defines the loss function we are trying to optimize
# mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

num_epochs = 500                  ############ please change this if you want to increase or decrease train time ############ 
for i in range(num_epochs):
    count = 0
    mean_loss = 0
    # Within each iteration, we will go over each minibatch of data
    minibatch_iter = train_loader
    for x_batch, y_batch in minibatch_iter:
        x_batch_enc = crypten.cryptensor(x_batch, src=ALICE)
        y_batch_enc = crypten.cryptensor(y_batch, src=ALICE)
        crypten_optimizer.zero_grad()
        output = model(x_batch_enc)
#         output_enc = crypten.cryptensor(output, src=ALICE)
        print('output', type(output))
        print('y_batch', type(y_batch))
        loss = -mll(output, y_batch_enc)
        print('loss', type(loss))
        loss.backward()
        crypten_optimizer.step()
        batch_loss = loss.get_plain_text()
        count+=1
        crypten.print(f"Loss {batch_loss.item():.4f}")
#     if i%10 == 0:
#       print("Mean Loss for Epoch", i,":", mean_loss/count)    

torch.float32
torch.float32
torch.int64
var torch.float32
torch.float32
MPCTensor(
	_tensor=0
	plain_text=HIDDEN
	ptype=ptype.arithmetic
)


RuntimeError: Autograd is not supported for in-place functions.

In [ ]:
# # Now we set both to training mode
# model.train()
# likelihood.train()

# # The use of an optimizer is again very similar to how it is used in neural nets. lr is learning rate.
# optimizer = torch.optim.Adam([
#     {'params': model.parameters()},
#     {'params': likelihood.parameters()},
# ], lr=0.05)

# # Our loss object. We're using the VariationalELBO. This defines the loss function we are trying to optimize
# mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data = train_y_enc.size(0))
# num_epochs = 500                  ############ please change this if you want to increase or decrease train time ############ 
# epochs_iter = range(num_epochs)

# # Define training parameters
# learning_rate = 0.001
# # num_epochs = 2
# batch_size = 200
# num_batches = train_x_enc.size(0) // batch_size

# rank = comm.get().get_rank()
# for i in range(num_epochs): 
#     crypten.print(f"Epoch {i} in progress:")       

#     for batch in range(num_batches):
#         # define the start and end of the training mini-batch
#         start, end = batch * batch_size, (batch + 1) * batch_size

#         # construct CrypTensors out of training examples / labels
#         x_train_ep = train_x_enc[start:end]
#         y_train_ep = train_y_enc[start:end]
# #         y_train = crypten.cryptensor(y_batch, requires_grad=True)

#         # perform forward pass:
#         output = model.forward(x_train_ep)
#         loss_value = -mll(output, y_train_ep)

# #         set gradients to "zero" (use any)
#         model.zero_grad()
# #         optimizer.zero_grad()

#         # perform backward pass: 
#         loss_value.backward()

#         # update parameters (use any)
#         model.update_parameters(learning_rate)
# #         optimizer.step()

#         # Print progress every batch:
#         batch_loss = loss_value.get_plain_text()
#         crypten.print(f"\tBatch {(batch + 1)} of {num_batches} Loss {batch_loss.item():.4f}")

In [ ]:
# model.decrypt()

In [ ]:
x_enc = crypten.cryptensor([1.0, 2.0, 3.0], [1.0, 2.0, 3.0])
y_enc = crypten.cryptensor([4.0, 5.0, 6.0], [4.0, 5.0, 6.0])


# Concatenation

z_enc = crypten.cat([x_enc, y_enc])
z_enc

In [ ]:
x_enc.cat([x_enc, y_enc, x_enc]).get_plain_text()

In [ ]:
x_enc.get_plain_text()